# Cryptography (CC4017) -- Week 10

## Exercise 1

Is (4,7) a point in the elliptic curve $y^2 = x^3−5x + 5$ over $\mathbb{Z}_{23} ?$ And over R?

In [6]:
def is_point_on_curve(x, y, p=None):
    # Check over R
    left_side_R = y**2
    right_side_R = x**3 - 5*x + 5
    on_curve_R = left_side_R == right_side_R

    # Check over Z_p
    if p is not None:
        left_side_Zp = (y**2) % p
        right_side_Zp = (x**3 - 5*x + 5) % p
        on_curve_Zp = left_side_Zp == right_side_Zp
    else:
        on_curve_Zp = None

    return on_curve_R, on_curve_Zp

x,y = 4,7
p = 23
on_curve_R, on_curve_Zp = is_point_on_curve(x, y, p)

print(f"Point ({x},{y}) is on the curve over R: {on_curve_R}")
print(f"Point ({x},{y}) is on the curve over Z_{p}: {on_curve_Zp}")

Point (4,7) is on the curve over R: True
Point (4,7) is on the curve over Z_23: True


## Exercice 2
On the elliptic curve real numbers $y^2=x^3−36x$, let $P=(−2,8)$ and $Q=(−3,−9)$.

Find $P+Q $ and $2P$.

In [7]:
def point_addition(P, Q):
    x1, y1 = P
    x2, y2 = Q
    
    if P == Q:
        return point_doubling(P)
    
    m = (y2 - y1) / (x2 - x1)
    x3 = m**2 - x1 - x2
    y3 = m * (x1 - x3) - y1
    
    return (x3, y3)

def point_doubling(P):
    x1, y1 = P
    
    m = (3 * x1**2 - 36) / (2 * y1)
    x3 = m**2 - 2 * x1
    y3 = m * (x1 - x3) - y1
    
    return (x3, y3)

# Example points
P = (-2, 8)
Q = (-3, -9)

# Calculate P + Q
R = point_addition(P, Q)
print("P + Q =", R)

# Calculate 2P
S = point_doubling(P)
print("2P =", S)

P + Q = (294.0, -5040.0)
2P = (6.25, 4.375)


## Exercice 3

Consider the elliptic curve defined by $y^2=x^3+x+6 $ over $\mathbb{Z}_{11}$. Determine all of the points of the curve.

In [8]:
def find_points_on_curve(p):
    points = []
    for x in range(p):
        rhs = (x**3 + x + 6) % p
        for y in range(p):
            if (y**2) % p == rhs:
                points.append((x, y))
    return points

# Define the prime modulus
p = 11

# Find all points on the elliptic curve y^2 = x^3 + x + 6 over Z_11
points = find_points_on_curve(p)

# Print the points
print("Points on the curve y^2 = x^3 + x + 6 over Z_11:")
for point in points:
    print(point)

# Include the point at infinity
print("Point at infinity: O")

Points on the curve y^2 = x^3 + x + 6 over Z_11:
(2, 4)
(2, 7)
(3, 5)
(3, 6)
(5, 2)
(5, 9)
(7, 2)
(7, 9)
(8, 3)
(8, 8)
(10, 2)
(10, 9)
Point at infinity: O


## Exercice 4

For the curve defined in the previous question, consider the point $G=(2,7)$.Compute the multiples of $G$ from $2G$ through $13G$.

In [9]:
def mod_inverse(a, p):
    """ Compute the modular inverse of a modulo p """
    return pow(a, p - 2, p)

def point_addition(P, Q, a, p):
    """ Add two points P and Q on the elliptic curve y^2 = x^3 + ax + b over Z_p """
    x1, y1 = P
    if P == Q:
        # Point doubling
        m = (3 * x1**2 + a) * mod_inverse(2 * y1, p) % p
        x2, y2 = x1, y1  # For consistency in the formula below
    else:
        # Point addition
        x2, y2 = Q
        m = (y2 - y1) * mod_inverse(x2 - x1, p) % p

    x3 = (m**2 - x1 - x2) % p
    y3 = (m * (x1 - x3) - y1) % p
    return (x3, y3)

def scalar_multiplication(k, P, a, b, p):
    """ Compute k * P on the elliptic curve y^2 = x^3 + ax + b over Z_p """
    Q = P
    R = None  # Point at infinity
    for i in range(k):
        if R is None:
            R = Q
        else:
            R = point_addition(R, Q, a, p)
    return R

# Define the elliptic curve parameters
a = 1
b = 6
p = 11
G = (2, 7)

# Compute multiples of G from 2G to 13G
multiples = []
for k in range(2, 14):
    multiples.append(scalar_multiplication(k, G, a, b, p))

# Print the multiples
print("Multiples of G = (2, 7) on the curve y^2 = x^3 + x + 6 over Z_11:")
for k, point in enumerate(multiples, start=2):
    print(f"{k}G = {point}")

Multiples of G = (2, 7) on the curve y^2 = x^3 + x + 6 over Z_11:
2G = (5, 2)
3G = (8, 3)
4G = (10, 2)
5G = (3, 6)
6G = (7, 9)
7G = (7, 2)
8G = (3, 5)
9G = (10, 9)
10G = (8, 8)
11G = (5, 9)
12G = (2, 4)
13G = (7, 7)


## Exercice 5 

Write python/SageMath programs that for P-192 and ECDSA

(a) Generates a pair of private/public keys.

(b) Sign a text using a private key.

(c) Verify the signature using the public key.

In [10]:
from ecdsa import SigningKey, NIST192p

# STEP 1: Generate a private key and public key
private_key = SigningKey.generate(curve=NIST192p)
public_key = private_key.get_verifying_key()

print("Private key:", private_key.to_string().hex())
print("Public key:", public_key.to_string().hex())

# STEP 2: Sign a message
message = b"Cripto is fun!"
signature = private_key.sign(message)

print("Signature:", signature.hex())

# STEP 3: Verify the signature
is_valid = public_key.verify(signature, message)

print("Signature is valid:", is_valid)

Private key: 81aa37953c80d285de6beedd4454020e1af3a2c097c4aeb0
Public key: f595c5e45a6719335ef50334a14cc7c26711d6d6ef92f48fd7dbd97cee55972eff5693836cd435ead2824b09bdf663b6
Signature: 5b5388e2b537cc8afa092b0a1ef37e4dcffd2eaa9be8259cbe60e45f70a884cb9f36f9b0bc08a6f01c7a898f69c55069
Signature is valid: True
